In [2]:
import random as rand
import bisect

In [223]:
## Hypergraph

def getRandomLevel(p = 0.5):
    lvl = 0
    while rand.random() < p and lvl < float('inf'):
        lvl = lvl + 1
    return lvl + 1

class HyperEdge:
    def __init__(self):
        self.nodes = []
        self.min = float("inf")
        self.max = float("-inf")
        self.height = 0

class HyperGraph:
    def __init__(self):
        self.p = 0.5
        self.height = getRandomLevel(self.p)
        self.hedges = [HyperEdge() for i in range(self.height + 1)]

    def lookup(self, nr):
        for he in self.hedges:
            if he.min <= nr <= he.max:
                L = len(he.nodes)
                i = bisect.bisect_left(he.nodes, nr)
                if i != L and he.nodes[i] == nr:
                    return True
        return False

    def findsert(self, nr, he):
        L = len(he.nodes)
        i = bisect.bisect_left(he.nodes, nr)
        if i != L and he.nodes[i] == nr:
            return False, False
        else:
            return i, he.nodes
    
    def insert(self, nr):

        height = getRandomLevel()

        if self.height < height:

            for i in range(height - self.height):
                self.hedges.append(HyperEdge())

            self.height = height

        test = self.hedges[height - 1]

        i, he = self.findsert(nr, test)

        if i is not False:
            he.insert(i, nr)
            if test.min > nr:
                test.min = nr
            if test.max < nr:
                test.max = nr
        else:
            return False

    def show_hedges(self):
        for i in self.hedges:
            print(i.nodes)

    def show_minmax(self):
        for i in self.hedges:
            print(f'min = {i.min} \nmax = {i.max}')

In [185]:
## Skiplist

class SkipNode:
    def __init__(self, height = 0, elem = None):
        self.elem = elem
        self.next = [None]*height

class SkipList:
    def __init__(self):
        self.head = SkipNode()

    def randomHeight(self):
        height = 1
        while rand.randint(1, 2) != 1:
            height += 1
        return height

    def updateList(self, elem):

        update = [None]*len(self.head.next)
        x = self.head

        for i in reversed(range(len(self.head.next))):
            while x.next[i] != None and \
                x.next[i].elem < elem:
                x = x.next[i]
            update[i] = x

        return update

    def find(self, elem, update = None):
        cnt = 0
        if update == None:
            update = self.updateList(elem)
        if len(update) > 0:
            candidate = update[0].next[0]
            if candidate != None and candidate.elem == elem:
                return candidate
        return None

    def findcnt(self, elem, update = None):
        cnt = 0
        if update == None:
            #update = self.updateList(elem)
            update = [None]*len(self.head.next)
            x = self.head
            for i in reversed(range(len(self.head.next))):
                while x.next[i] != None and x.next[i].elem < elem:
                    cnt = cnt + 1
                    x = x.next[i]
                update[i] = x
            
        cnt = cnt + 1
        if len(update) > 0:
            candidate = update[0].next[0]
            cnt = cnt + 1
            if candidate != None and candidate.elem == elem:
                cnt = cnt + 1
                return cnt
        return cnt
    
    def insert(self, elem):

        node = SkipNode(self.randomHeight(), elem)

        while len(self.head.next) < len(node.next):
            self.head.next.append(None)

        update = self.updateList(elem)
        if self.find(elem, update) == None:
            for i in range(len(node.next)):
                node.next[i] = update[i].next[i]
                update[i].next[i] = node

In [177]:
# SkipList Naive
class NodeSKP(object): 
    def __init__(self, key, level): 
        self.key = key 
        self.forward = [None]*(level+1) 
  
class SkipListNaive(object): 
    def __init__(self, max_lvl, P): 
        self.MAXLVL = max_lvl 
        self.P = P 
        self.header = self.createNode(self.MAXLVL, -1) 
        self.level = 0
      
    def createNode(self, lvl, key): 
        n = NodeSKP(key, lvl) 
        return n 

    def randomLevel(self): 
        lvl = 0
        while rand.random()<self.P and lvl<self.MAXLVL:lvl += 1
        return lvl 

    def find(self, key):  
        current = self.header 
        for i in range(self.level, -1, -1): 
            while(current.forward[i] and current.forward[i].key < key): 
                current = current.forward[i] 
        current = current.forward[0] 
        if current and current.key == key: 
            return key  

    def insert(self, key): 
        update = [None]*(self.MAXLVL+1) 
        current = self.header 
        for i in range(self.level, -1, -1): 
            while current.forward[i] and current.forward[i].key < key: 
                current = current.forward[i] 
            update[i] = current 
            
        current = current.forward[0] 
  
        if current == None or current.key != key: 
            rlevel = self.randomLevel() 
  
            if rlevel > self.level: 
                for i in range(self.level+1, rlevel+1): 
                    update[i] = self.header 
                self.level = rlevel 
  
            n = self.createNode(rlevel, key) 
  
            for i in range(rlevel+1): 
                n.forward[i] = update[i].forward[i] 
                update[i].forward[i] = n 

In [187]:
class Node:
    def __init__(self, val):
        self.l = None
        self.r = None
        self.v = val

class Tree:
    def __init__(self):
        self.root = None

    def getRoot(self):
        return self.root

    def add(self, val):
        if self.root is None:
            self.root = Node(val)
        else:
            self._add(val, self.root)

    def _add(self, val, node):
        if val < node.v:
            if node.l is not None:
                self._add(val, node.l)
            else:
                node.l = Node(val)
        else:
            if node.r is not None:
                self._add(val, node.r)
            else:
                node.r = Node(val)

    def find(self, val):
        if self.root is not None:
            return self._find(val, self.root)
        else:
            return None

    def _find(self, val, node):
        if val == node.v:
            return node
        elif (val < node.v and node.l is not None):
            self._find(val, node.l)
        elif (val > node.v and node.r is not None):
            self._find(val, node.r)

    def deleteTree(self):
        # garbage collector will do this for us.
        self.root = None

    def printTree(self):
        if self.root is not None:
            self._printTree(self.root)

    def _printTree(self, node):
        if node is not None:
            self._printTree(node.l)
            print(str(node.v) + ' ')
            self._printTree(node.r)

In [226]:
## Insert timeit

hgraph = HyperGraph()
hgraph.p = 0.5
btree = Tree()
skeep = SkipList()
skeepNaive = SkipListNaive(10, 0.5)
#llist = LinkedList()
#sset = set()
#bset = 0

nr = 500000

ten_thousand_integers = [rand.randint(1, 5000000) for i in range(nr)]

def ten_k_hg(HG):
    for i in range(nr):
        HG.insert(ten_thousand_integers[i])

def ten_k_sl(skeep):
    for i in range(nr):
        skeep.insert(ten_thousand_integers[i])

def ten_k_sln(skeepNaive):
    for i in range(nr):
        skeepNaive.insert(ten_thousand_integers[i])
        
def ten_k_liszt(llist):
    for i in range(nr):
        llist.insert(ten_thousand_integers[i])

def ten_k_btree(btree):
    for i in range(nr):
        btree.add(ten_thousand_integers[i])

def ten_k_sset(sset):
    for i in range(nr):
        sset.add(ten_thousand_integers[i])
        
def ten_k_bset(bset):
    for i in range(nr):
        bset |= 1 << i
# So far it's 10 times slower than a skip list
# But 8 times faster than a linked list.
#%timeit -r 1 -n 10 ten_k_sset(sset)
#%timeit -r 1 -n 10 ten_k_bset(bset)
%timeit -r 1 -n 1 ten_k_hg(hgraph)
%timeit -r 1 -n 1 ten_k_sl(skeep)
%timeit -r 1 -n 1 ten_k_sl(skeepNaive)
%timeit -r 1 -n 1 ten_k_btree(btree)
#%timeit -r 1 -n 10 ten_k_liszt(llist)
#%timeit -r 1 -n 10 sorted(ten_thousand_integers)

14.9 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
9.59 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
15.4 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
4.09 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [163]:
def binarySearch(arr, l, r, x):

    while l <= r:

        mid = l + (r - l) // 2

        # Check if x is present at mid
        if arr[mid] == x:
            return mid

        # If x is greater, ignore left half
        elif arr[mid] < x:
            l = mid + 1

        # If x is smaller, ignore right half
        else:
            r = mid - 1

    # If we reach here, then the element
    # was not present
    return -1

In [227]:
## Find timeit
listhg = []
listskp = []
liszt = sorted(ten_thousand_integers)

def ten_k_hg(HG):
    for i in range(nr):
        HG.lookup(ten_thousand_integers[i])

def ten_k_sl(skeep):
    for i in range(nr):
        skeep.findcnt(ten_thousand_integers[i])        
        
def ten_k_sl_naive(skeepNaive):
    for i in range(nr):
        skeepNaive.find(ten_thousand_integers[i])

def ten_k_liszt(llist):

    for i in range(nr):
        llist.find(ten_thousand_integers[i])

def ten_k_sset(sset):
    for i in range(nr):
        ten_thousand_integers[i] in sset
        
def ten_k_bset(bset):
    for i in range(nr):
        bset & 1 << ten_thousand_integers[i]

def ten_k_btree(btree):
    for i in range(nr):
        btree.find(ten_thousand_integers[i])

def ten_k_binarySearch(llist):
    for i in range(nr):
        binarySearch(llist, 0, len(llist), ten_thousand_integers[i])

#%timeit -r 1 -n 10 ten_k_bset(bset)
#%timeit -r 1 -n 10 ten_k_sset(sset)
%timeit -r 1 -n 1 ten_k_hg(hgraph)
%timeit -r 1 -n 1 ten_k_sl(skeep)
%timeit -r 1 -n 1 ten_k_sl_naive(skeepNaive)
%timeit -r 1 -n 1 ten_k_btree(btree)
%timeit -r 1 -n 1 ten_k_binarySearch(liszt)
#%timeit -r 1 -n 10 ten_k_liszt(llist)
#%timeit -r 1 -n 10 sorted(ten_thousand_integers)

### So apparently the lookup function is slow asf.
### Insert is actually kinda ok.

2.34 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
8.72 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
30.7 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
5.22 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
2.78 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [30]:
#import pickle
#import sys
#sys.setrecursionlimit(100000)

#pickle.dump(hgraph, open("hgraph.p", "wb" ))
#pickle.dump(skeep, open("skeep.p", "wb"))

In [213]:
hgraph.show_hedges()
hgraph.show_minmax()

hgraph.lookup(303)

[77, 246, 277, 334, 461, 519, 521, 613, 682, 917, 1083, 1202, 1225, 1270, 1495, 1678, 1712, 1729, 1871, 2102, 2116, 2261, 2283, 2433, 2518, 2708, 2710, 2724, 2896, 3027, 3105, 3284, 3609, 3699, 3773, 3858, 3909, 3915, 4256, 4266, 4276, 4286, 4456, 4499, 4531, 4599, 4768, 4781, 5028, 5090, 5157, 5222, 5294, 5299, 5316, 5395, 5901, 5924, 6048, 6067, 6098, 6109, 6318, 6525, 6607, 6777, 6911, 7067, 7123, 7148, 7279, 7312, 7360, 7673, 7708, 7771, 7869, 8096, 8118, 8241, 8243, 8251, 8275, 8466, 8472, 8513, 8549, 8609, 8653, 8686, 8771, 8782, 8872, 8880, 8890, 8984, 9043, 9299, 9338, 9359, 9658, 9660, 9674, 9730, 10010, 10220, 10251, 10696, 10750, 10854, 10867, 10881, 10901, 10903, 11063, 11158, 11262, 11338, 11410, 11416, 11495, 11506, 11573, 11606, 11611, 11709, 11730, 11749, 11758, 11815, 11904, 11944, 12015, 12106, 12192, 12248, 12379, 12425, 12532, 12614, 12621, 12636, 12811, 12852, 12877, 12904, 12982, 13118, 13134, 13221, 13299, 13332, 13460, 13486, 13497, 13506, 13583, 13660, 13690, 1

False